# Lab 9 Decision Trees vs otros classifiers

In [ ]:
import pandas as pd #panda dataframes

Location = r'Tabla_impago_full.xlsx'

##### Importo tabla 

In [ ]:
Tabla1 = pd.read_excel(Location,sheet_name="Tabla_B")

##### Leo tabla ignoro los IDs

In [ ]:
data = Tabla1.iloc[:,2:]
data=data.fillna(data.mean())

data1=data.iloc[:,0:2]

##### Separo la variable dependiente de las explicativas

In [ ]:
Xs=data.iloc[:,1:]
Y=data.iloc[:,0]

### Separo mi base de datos en dos muestras, una muestra de entrenamiento y una de prueba

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Xs, Y, test_size=0.5, random_state=42)

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score


clf = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1)

clf.fit(X_train,y_train)

tree.export_graphviz(clf,out_file='tree.dot')

## se puede visualizar copiando y pegando el contenido del archivo .dot en http://www.webgraphviz.com/

##### Importancia de las variables en el árbol

In [ ]:
feature_importance=clf.feature_importances_

### Predicciones y probabilidades en la muestra de entrenamiento

In [ ]:
pred_train=clf.predict(X_train)              #predicción
probs_train=clf.predict_proba(X_train)          #probabilidad

##### Qué regla o nodo se activa para cada predicción?

In [ ]:
rules=clf.apply(X_train)

### Utilizando esta función puedo escribir el árbol como reglas de decisión (lenguaje humano)

In [ ]:
from sklearn.tree import _tree
def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print "def tree({}):".format(", ".join(feature_names))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print "{}if {} <= {}:".format(indent, name, threshold)
            recurse(tree_.children_left[node], depth + 1)
            print "{}else:  # if {} > {}".format(indent, name, threshold)
            recurse(tree_.children_right[node], depth + 1)
        else:
            print "{}return {}".format(indent, tree_.value[node])

    recurse(0, 1)

# la función debe ejecutarse desde el terminal: tree_to_code(clf,X_train.columns)

### Calculemos el ajuste accuracy del modelo en train y en test

In [ ]:
probs2=clf.predict_proba(X_test)

##### Una vez que ya tengo las predicciones puedo calcular matriz de confusión, accuracy, recall, precision, auc, etc etc

In [ ]:
score_train = clf.score(X_train, y_train) 
score_testing = clf.score(X_test, y_test) 

##### Para calcular otras métricas precision, recall, etc necesito también predecir en testing  

In [ ]:
pred_testing=clf.predict(X_test)               #predigo en base testing

##### Calculo recall, precision, auc, fpr, tpr, auc, f1 etc en ambas bases

In [ ]:
fpr_train, tpr_train, thresholds_train = metrics.roc_curve(y_train, pred_train, pos_label=1)
fpr_test, tpr_test, thresholds_test = metrics.roc_curve(y_test, pred_testing, pos_label=1)

auc_train=metrics.auc(fpr_train, tpr_train)
auc_test=metrics.auc(fpr_test, tpr_test)

recalls_train=recall_score(y_train, pred_train, average=None)
recalls_test=recall_score(y_test, pred_testing, average=None)

precis_train=precision_score(y_train, pred_train, average=None)
precis_test=precision_score(y_test, pred_testing, average=None)

f1sc_train=f1_score(y_train, pred_train, average=None)
f1sc_test=f1_score(y_test, pred_testing, average=None) 